In [33]:
import heapq

class NetworkNode:
    def __init__(self, ID, latitude, longitude):
        self.ID = ID
        self.latitude = latitude
        self.longitude = longitude

    def __repr__(self):
        return f"{self.__class__.__name__}({self.ID})"

    def __hash__(self):
        return hash(self.ID)

    def __eq__(self, other):
        return self.ID == other.ID

# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
class FogCloudNode(NetworkNode):
    def __init__(self, ID, longitude, latitude, processing_capacity, memory_capacity, cost, model):
        super().__init__(ID, latitude, longitude)
        self.model = model
        self.processing_capacity = processing_capacity
        self.memory_capacity = memory_capacity
        self.cost = cost

class FogNode(FogCloudNode):
    pass

class CloudNode(FogCloudNode):
    pass

# Sensores -> index | longitude | latitude | services
class Sensor(NetworkNode):
    def __init__(self, ID, longitude, latitude, services):
        super().__init__(ID, latitude, longitude)
        self.services = services

class Grafo:
    def __init__(self):
        self.adj = {}
        self.n_vertices = 0
        self.n_arestas = 0

    def add_vertice(self, noh):
        if noh not in self.adj:
            self.adj[noh] = []
            self.n_vertices += 1

    def add_aresta(self, u, v, largura_banda, custo, dist_H):
        self.add_vertice(u)
        self.add_vertice(v)
        # Tempo de delay = latência (valor obtido da tecnologia) + propagação (harvesine/2*10^8)
        tempo = 0.001 + dist_H/(2*10**8)
        self.adj[u].append((v, largura_banda, custo, tempo))
        self.n_arestas += 1

def dijkstra(grafo, origem, atributos_req):
    tempo = {n: float('inf') for n in grafo.adj}
    tempo[origem] = 0
    prev = {n: None for n in grafo.adj}
    fila = [(0, origem)]

    while fila:
        tempo_atual, u = heapq.heappop(fila)
        if tempo_atual > tempo[u]:
            continue

        for v, _, _, t in grafo.adj[u]:
            novo_tempo = tempo[u] + t 
            if novo_tempo < tempo[v]:
                tempo[v] = novo_tempo
                prev[v] = u
                heapq.heappush(fila, (novo_tempo, v))

    return tempo, prev

def reconstruir_caminho(prev, destino):
    caminho = []
    while destino is not None:
        caminho.append(destino)
        destino = prev[destino]
    return caminho[::-1]


# Criando os nós
# Sensores -> index | longitude | latitude | services
# Fog -> index | longitude | latitude | processing capacity | memory capacity | cost | model
# Cloud -> index | longitude | latitude | processing capacity | memory capacity | cost | model
sensor = Sensor("S1", 0.0, 0.0, services=[])
fog1 = FogNode("F1", 1.0, 1.0, 80, 16, 5, "model-A")
fog2 = FogNode("F2", 1.5, 1.5, 70, 8, 6, "model-B")
cloud = CloudNode("C1", 2.0, 2.0, 500, 128, 15, "cloud-X")

# Criando o grafo e adicionando conexões
g = Grafo()

# Arestas -> node i | node j | bandwidth i-j | bandwidth cost (US$/Gbps) | haversine distance i-j
g.add_aresta(sensor, fog1, 3, 4, 12)     
g.add_aresta(sensor, fog2, 5, 6, 30)     
g.add_aresta(fog1, fog2, 4, 5, 40)       
g.add_aresta(fog2, fog1, 6, 5, 12)      
g.add_aresta(fog1, cloud, 4, 8, 90)      
g.add_aresta(fog2, cloud, 4, 4, 120)      

# ID: [processing_demand, memory_demand, number_of_bits, lifetime]
services = {
    'waste': [0.2125, 0.375, 296, 50],
    'camera': [0.35, 0.475, 12000, 10],
    'air': [0.25, 0.3125, 744, 100]
}

tempo, prev = dijkstra(g, sensor, services["waste"])
print(tempo)
print(prev)
print(reconstruir_caminho(prev, cloud))

# Próximos passos
# Loop: grafo -> recebe requisição -> calcula dijkstra -> processa requisição no caminho retornado -> atualiza grafo (residual)
# Jeito rápido de atualizar o grafo



{Sensor(S1): 0, FogNode(F1): 0.00100006, FogNode(F2): 0.00100015, CloudNode(C1): 0.00200051}
{Sensor(S1): None, FogNode(F1): Sensor(S1), FogNode(F2): Sensor(S1), CloudNode(C1): FogNode(F1)}
[Sensor(S1), FogNode(F1), CloudNode(C1)]
